In [1]:
import numpy as np
from torch.utils.data import DataLoader

from utils import CustomDataset2, confusion_matrix, dice_coeff_batch, DiceLoss
from natsort import natsorted
from glob import glob
from unet import AttU_Net
from tqdm import tqdm

import numpy as np
import logging
import os, sys
import random
import torch
import time
import cv2

from PIL import Image
#import matplotlib.pyplot as plt

In [2]:
from glob import glob

#slname = ""
main_data_dir = '/Users/anuradha.kar/Documents/python_scripts/docker_tests/docker_py1/src/'
inf_dir = 'images' ## 132 slide

#############
pin_memory= True
scale_factor = 1.0
n_input_channels = 3
inf_imgs_fold = []
batch_size = 1
n_devices = 1
n_workers = 0
tmp_inf_imgs = natsorted(glob(os.path.join(main_data_dir,inf_dir,'*.png')))

               
for s in range(len(tmp_inf_imgs)):
    inf_imgs_fold.append(tmp_inf_imgs[s])
                    
inf_dataset = CustomDataset2(inf_imgs_fold, normalize=False,cached_data=pin_memory, n_channels=n_input_channels,scale=scale_factor)
inf_loader = DataLoader(inf_dataset, batch_size=batch_size*n_devices, shuffle=False, pin_memory=False, num_workers=n_workers)

          

100%|██████████████████████████████████████████| 50/50 [00:00<00:00, 394.80it/s]


In [3]:
from unet.unet_model import AttU_Net

n_input_channels = 3

    # The number of output classes  (N classes  ->  n_output_channels = N)
n_output_channels = 1
    #######################################################################
    
    # defining the U-Net model
model = AttU_Net(img_ch=n_input_channels, output_ch=n_output_channels)

In [4]:
# Chose the GPU CUDA devices to make the training go much faster vs CPU use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# For faster convolutions, but more memory
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

#model = Unet(inchannels=3, outchannels=1, net_depth=4)
# Putting the model inside the device
model.to(device=device, dtype=torch.float32)

MODEL_PATH = '/Users/anuradha.kar/Documents/python_scripts/unet-pytorch/test_set/best_model_bg10.pth'
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))

#model.eval()

<All keys matched successfully>

In [5]:
kernel = np.ones((3, 3), np.uint8)
ct= 0
ps = 128 
all_poly= []
for batch in tqdm(inf_loader):
    image = batch['image']
    fname = tmp_inf_imgs[ct]
    fname = os.path.basename(os.path.normpath(fname))
    
    image = image.to(device=device, dtype=torch.float32)
    
    
    with torch.no_grad():
        pred_mask = model(image)
        pred = torch.sigmoid(pred_mask)
        pred = (pred > 0.95).float()
        pred = pred.cpu()
   
    pred = Image.fromarray(np.uint8(np.squeeze(pred)*255))
    pred= np.asarray(pred)
    pred= cv2.dilate(pred, kernel, iterations=1)
    
    ###### create polygons here
    fname= fname.replace('.png','')
    x1, y1, x2, y2= float(fname.split('_')[1]), float(fname.split('_')[2]),float(fname.split('_')[3]),float(fname.split('_')[4])
    a= (x2-x1)/2
    b= (y2-y1)/2
    x3= x1 - 64+a  ##bottom left corner of patch
    y3= y1- 64+b
    x4= x3  ## top left corner which is the origin (0,0) for the patch coordinates
    y4= y3-ps
    
    ret, thresh = cv2.threshold(pred, 127, 255, cv2.THRESH_BINARY)
    
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    polygons = []
    
    #############################################
    
    for object in contours:
        coordsx = []
        coordsy = []
        for point in object:
            xval= float(point[0][0])+ float(x4)
            yval=float(point[0][1])+ float(y4)+128 ##128 added for shift
            coordsx.append(xval) ## add x directly here
            coordsy.append(yval) ## add y directly here
        coords= list(zip(coordsx, coordsy))
        poly=np.array(coords)
        if len(poly)> 30:   ##if polygon has more than 10 points``
            polygons.append(np.array(coords))
    all_poly.append(polygons) 
      
    #cv2.imwrite('/Users/anuradha.kar/Documents/python_scripts/docker_tests/docker_py1/src/preds/'+ fname, pred)
    ct+=1
 

100%|███████████████████████████████████████████| 50/50 [00:16<00:00,  2.99it/s]


In [ ]:

from shapely.geometry import Polygon
polygon_patch1= Polygon(all_poly[20][0]) ## 1st id- id image, 2nd- id annotation
polygon_patch1

In [8]:
## connect to cytomine
## print stuff from cytomine

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from cytomine import Cytomine
#from cytomine.models import *
import numpy as np
#from skimage import io
import string
from operator import sub
import os

from cytomine.models import ProjectCollection
#from cytomine.models import AnnotationCollection, ImageInstanceCollection
from cytomine.models import StorageCollection

import logging
import sys
from argparse import ArgumentParser

import os

from shapely import wkt
from shapely.affinity import affine_transform
#from cytomine.models import AnnotationCollection, ImageInstanceCollection
from cytomine.models import Property, Project, Annotation, ImageInstance
from cytomine.models import Property, Annotation, AnnotationTerm, AnnotationCollection
from shapely.geometry import Polygon, LineString
from cytomine.models import AnnotationCollection, ImageInstanceCollection
from cytomine.models.ontology import Ontology, Term, RelationTerm, TermCollection

In [ ]:
from cytomine.models import Project
from cytomine import Cytomine
from cytomine.models import CurrentUser


#guest public key 184dd9ed-a12a-4687-9361-f953aa52ba75
#guest private key 938115a5-d7ee-4834-9344-20344e7e8a11
#a40ce3ac-a46c-4018-8d29-a3906121a9c9'
#c662cb38-95f3-4fbd-9253-3b79a90667ce'
#host = 'https://demo.cytomine.com'#https://cytomine-staging.icm-institute.org'#'https://demo.cytomine.com'
#public_key = 'e15eeae2-6203-483b-92b4-9d95dab8e48f'
#private_key = 'a1f2ef5f-cb66-4082-b877-2041bd7d53d8'

host = 'https://cytomine-staging.icm-institute.org'#https://cytomine-staging.icm-institute.org'#'https://demo.cytomine.com'
public_key = 'a40ce3ac-a46c-4018-8d29-a3906121a9c9' 
private_key = 'c662cb38-95f3-4fbd-9253-3b79a90667ce'

with Cytomine(host, public_key, private_key) as cytomine:
    # We are connected !
    me = CurrentUser().fetch()

    # It will print your username, that has been retrieved from Cytomine.
    print(me.username)

projects = ProjectCollection().fetch()
print(projects)
for project in projects:
    
    print("Project name: ", project.name, "Project id: ", project.id)

    image_instances = ImageInstanceCollection().fetch_with_filter("project", 5923)
print(image_instances)

for image in image_instances:
    print("Image ID: {} | Width: {} | Height: {} | Resolution: {} | Filename: {}".format(
                image.id, image.width, image.height, image.magnification, image.filename))

terms = TermCollection().fetch_with_filter("project", 5923)
for term in terms:
    print("Term ID: {} | Name: {} | Color: {}".format(term.id, term.name, term.color))
    ## create pandas dataframe


In [12]:
from cytomine.models import Project
from cytomine import Cytomine
from cytomine.models import CurrentUser

id_image=480387
id_term= 360612
simplify_tolerance = 5

host = 'https://cytomine-staging.icm-institute.org'#https://cytomine-staging.icm-institute.org'#'https://demo.cytomine.com'
public_key = 'a40ce3ac-a46c-4018-8d29-a3906121a9c9' 
private_key = 'c662cb38-95f3-4fbd-9253-3b79a90667ce'

from shapely.geometry import Polygon

with Cytomine(host, public_key, private_key) as cytomine:
    # We are connected !
    me = CurrentUser().fetch()

    # It will print your username, that has been retrieved from Cytomine.
    uname= str(me.username)


for i in range(1,len(all_poly)):
    if all_poly[i]!= []:
        polygon_patch1= Polygon(all_poly[i][0])
        polygon_patch1 = polygon_patch1.simplify(tolerance=simplify_tolerance)
        annotation_poly1 = Annotation(location=polygon_patch1.wkt, id_image=id_image).save()
        AnnotationTerm(annotation_poly1.id, id_term).save()

INFO:cytomine.client:[GET] /server/ping | 200 
INFO:cytomine.client:[GET] [currentuser] CURRENT USER - 4590 : anuradha | 200 
INFO:cytomine.client:[GET] [currentuser] CURRENT USER - 4590 : anuradha | 200 
INFO:cytomine.client:[POST] [annotation] 489263 | 200 
INFO:cytomine.client:[POST] [annotationterm] Annotation 489263 - Term 360612 | 200 
INFO:cytomine.client:[POST] [annotation] 489276 | 200 
INFO:cytomine.client:[POST] [annotationterm] Annotation 489276 - Term 360612 | 200 
INFO:cytomine.client:[POST] [annotation] 489291 | 200 
INFO:cytomine.client:[POST] [annotationterm] Annotation 489291 - Term 360612 | 200 
INFO:cytomine.client:[POST] [annotation] 489304 | 200 
INFO:cytomine.client:[POST] [annotationterm] Annotation 489304 - Term 360612 | 200 
INFO:cytomine.client:[POST] [annotation] 489317 | 200 
INFO:cytomine.client:[POST] [annotationterm] Annotation 489317 - Term 360612 | 200 
INFO:cytomine.client:[POST] [annotation] 489330 | 200 
INFO:cytomine.client:[POST] [annotationterm] 